In [1]:
x = {"swap": "hi"}

In [3]:
if "send" in x and "contract" in x["send"] and x["send"]["contract"] == "hi":
    print("damn thats wild")

In [1]:
16_623_718 - 9_066_108

7557610

In [2]:
1_999_268_640 - 7557610

1991711030

In [1]:
import httpx
import json
import time
from base64 import b16encode, b64decode
from cosmpy.protos.cosmwasm.wasm.v1.query_pb2 import (
    QuerySmartContractStateRequest,
    QuerySmartContractStateResponse)

def junoswap_info(rpc_url: str, contract_address: str) -> dict:
    """Given a JunoSwap contract address, and an rpc url,
       Query the node for the contract info, decode the 
       Response, and return the pool info as a dict

    Args:
        rpc_url (str): The rpc url of the node to query
        contract_address (str): The JunoSwap contract address

    Returns:
        dict: The contract info
    """
    data = QuerySmartContractStateRequest.SerializeToString(
                QuerySmartContractStateRequest(address=contract_address, 
                     query_data=json.dumps({"info":{}}).encode('utf-8')))
    payload = [{
        "jsonrpc": "2.0",
        "id": 1,
        "method": "abci_query",
        "params": {"path": "/cosmwasm.wasm.v1.Query/SmartContractState", 
                   "data": b16encode(data).decode("utf-8"), "prove": False}},
        {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "abci_query",
        "params": {"path": "/cosmwasm.wasm.v1.Query/SmartContractState", 
                   "data": b16encode(data).decode("utf-8"), "prove": False}}]

    return httpx.post(rpc_url, json=payload)

    """
    async with httpx.AsyncClient() as client:
        response = await client.post(rpc_url, json=payload)
    return response.json()
    val = b64decode(response.json()["result"]["response"]["value"])
    pool_info = json.loads(QuerySmartContractStateResponse.FromString(val).data.decode())
    return pool_info
    """

In [30]:
start = time.time()
x = junoswap_info("https://rpc-juno-ia.cosmosia.notional.ventures/", "juno1qt7uzjg9su3mk78jpt695rmxce4sa7evz7wa0edexjrsxghy35hsgje5l9")
end = time.time()
print(end - start)

0.7295749187469482


In [27]:
x

<coroutine object junoswap_info at 0x106df35a0>

In [23]:
len(x)

69

In [12]:
x[1]

{'jsonrpc': '2.0',
 'id': 1,
 'result': {'response': {'code': 0,
   'log': '',
   'info': '',
   'index': '0',
   'key': None,
   'value': 'CuUCeyJ0b2tlbjFfcmVzZXJ2ZSI6IjE1NjQ2NTg2ODEiLCJ0b2tlbjFfZGVub20iOnsiY3cyMCI6Imp1bm8xcXNyZXJjcWVndnM0eWUweXFnOTNrbnY3M3llNWRjM3BycXdkNmpjZGN1ajhnZ3A2dzB1czY2ZGV1cCJ9LCJ0b2tlbjJfcmVzZXJ2ZSI6IjQwMDMzNjc5NjIiLCJ0b2tlbjJfZGVub20iOnsiY3cyMCI6Imp1bm8xNXUzZHQ3OXQ2c3h4YTN4M2twa2h6c3k1NmVkYWE1YTY2d3Z0M2t4bXVrcWp6MnN4MGhlczVzbjM4ZyJ9LCJscF90b2tlbl9zdXBwbHkiOiIxMDExMzI5MjQ2IiwibHBfdG9rZW5fYWRkcmVzcyI6Imp1bm8xczZ2bnpmOTRkNXNhMGF4NzM1eTN1ZzRzdm1nNXBjNDBsdW1zcXlna2M1anhuZ3djY3Fsczd6NXhqdiJ9',
   'proofOps': None,
   'height': '5754332',
   'codespace': ''}}}

In [1]:
from calculate import calculate_swap, calculate_terraswap_swap

In [2]:
# Skip helper library Import
import skip

# General Imports
import json
import aiometer
import asyncio
import time
import anyio
import httpx
import logging
import requests

from hashlib import sha256

# Crypto/Cosmos Imports
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins
from cosmpy.aerial.client import LedgerClient, NetworkConfig
from cosmpy.aerial.contract import LedgerContract
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

# Imports from the files in the repo
from query_contract_info import junoswap_info, terraswap_info
from mempool import check_for_swap_txs_in_mempool
from update_reserves import update_pool, update_pools
from swaps import Swap, PassThroughSwap
from calculate import check_no_arbitrage_condition, calculate_optimal_amount_in, get_profit_from_route
from create_tx import create_tx
from messages import create_route_msgs, send, junoswap_swap
from route import get_route_object
# All global variables to be used throughout the program

# Mnenomic to generate private key
# Replace with your own mnemonic
MNEMONIC = "arm all mouse project cube mimic seminar year appear person world coconut"

# RPC URL to stream mempool and query contract state from
RPC_URL = "https://rpc-juno-ia.cosmosia.notional.ventures/"

# Rest URL to use to get a network client from cosmpy 
REST_URL = "https://api-juno-ia.cosmosia.notional.ventures/"

# Chain ID of network, to be used for network client
# You can find chain IDs at https://github.com/cosmos/chain-registry
CHAIN_ID = "juno-1"

# Fee asset denom to for network, to be used for network client
FEE_DENOM = "ujuno"

# Total gas limit for transaction
# This can be optimized in the future to be dynamic
# Based on the number of messages in the transaction
GAS_LIMIT = 1000000

# Price per gast unit to calculate total fee paid for has
GAS_PRICE = 0.001

# Gas fee to be paid for transaction
# Calculated by multiplying gas limit and gas price
GAS_FEE = int(GAS_LIMIT * GAS_PRICE)

# Addrex prefix for the network, to be used to generate wallet object
ADDRESS_PREFIX = "juno"

# RPC URL to send skip bundle to
# Can find the respective url at: 
# https://skip-protocol.notion.site/Skip-Configurations-By-Chain-a6076cfa743f4ab38194096403e62f3c
SKIP_RPC_URL = "http://juno-1-api.skip.money/"

# Address to send bid payment to for skip's blockspace auction
AUCTION_HOUSE_ADDRESS = "juno10g0l3hd9sau3vnjrayjhergcpxemucxcspgnn4"

# Auction bid amount
# This can be optimized in the future to be dynamic
# Based on profit of the transaction
AUCTION_BID = 1000

cfg = NetworkConfig(
    chain_id=CHAIN_ID,
    url=f"rest+{REST_URL}",
    fee_minimum_gas_price=GAS_PRICE,
    fee_denomination=FEE_DENOM,
    staking_denomination=FEE_DENOM,
)
client = LedgerClient(cfg)

In [4]:
# Get wallet object from mnemonic seed phrase
seed_bytes = Bip39SeedGenerator(MNEMONIC).Generate()
bip44_def_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.COSMOS).DeriveDefaultPath()
wallet = LocalWallet(PrivateKey(bip44_def_ctx.PrivateKey().Raw().ToBytes()), prefix=ADDRESS_PREFIX)

In [3]:
msg_send = send("juno1zhqrfu9w3sugwykef3rq8t0vlxkz72vwnnptts", AUCTION_HOUSE_ADDRESS, 10000, 'ujuno')

In [13]:
tx_bytes, tx = create_tx(client, wallet, [msg_swap, msg_swap, msg_send], GAS_LIMIT, GAS_FEE, "ujuno")

In [12]:
msg_swap = junoswap_swap(wallet, "juno10g0l3hd9sau3vnjrayjhergcpxemucxcspgnn4", "Token1", 1000000, "ujuno", 100000)

In [16]:
tx_bytes, tx = create_tx(client, wallet, [msg_swap, msg_swap, msg_swap, msg_swap, msg_send], GAS_LIMIT, GAS_FEE, "ujuno")
len(tx_bytes)

1275

In [14]:
len(tx_bytes)

795

In [17]:
query={"simulation": 
        {"offer_asset": 
            {"amount": "10000000", 
            "info": 
                {"native_token": {"denom": "ujuno"}}}}}

In [4]:
pair_contract = LedgerContract(
    path=None, client=client, address="juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07"
)

In [18]:
pair_contract.query({"pool": {}})

{'assets': [{'info': {'token': {'contract_addr': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup'}},
   'amount': '1635544563657'},
  {'info': {'native_token': {'denom': 'ujuno'}}, 'amount': '18055283661'}],
 'total_share': '166714201682'}

In [16]:
pair_contract.query({"pool": {}})

{'assets': [{'info': {'token': {'contract_addr': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup'}},
   'amount': '1635544563657'},
  {'info': {'native_token': {'denom': 'ujuno'}}, 'amount': '18055283661'}],
 'total_share': '166714201682'}

In [19]:
pair_contract.query(query)

{'return_amount': '902726752',
 'spread_amount': '501434',
 'commission_amount': '2625521'}

In [20]:
calculate_terraswap_swap(18055283661, 1635544563657, 10000000, 0.0029)

0.9971


(902726751, 18065283661, 1634641836906, 2625521.5917000123)

In [21]:
calculate_swap(18055283661, 1635544563657, 10000000, 0.0029)

(902728201, 18065283661, 1634641835456)

In [4]:
import json

from base64 import b64decode

from query_contract_info import junoswap_info, terraswap_info
from mempool import check_for_swap_txs_in_mempool
from update_reserves import update_pool, update_pools
from swaps import Swap, PassThroughSwap

from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins
from cosmpy.aerial.client import LedgerClient, NetworkConfig
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

import aiometer
from functools import partial

import cosmpy.protos.cosmos.tx.v1beta1.tx_pb2 as cosmos_tx_pb2
import cosmpy.protos.cosmwasm.wasm.v1.tx_pb2 as cosmwasm_tx_pb2

In [3]:
bytes = b"0AF50A0A89010A1C2F636F736D6F732E62616E6B2E763162657461312E4D736753656E6412690A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070122B6A756E6F313067306C3368643973617533766E6A7261796A68657267637078656D756378637370676E6E341A0D0A05756A756E6F1204313030300AF8010A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E747261637412CF010A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070123F6A756E6F31323464307A796D726B64787637326363797571727175757238646B65736D786D7832756E666E3764656A393579717835796E387337307833796A1A4C7B2273776170223A7B22696E7075745F746F6B656E223A22546F6B656E31222C22696E7075745F616D6F756E74223A223335383031373537222C226D696E5F6F7574707574223A2231227D7D2A110A05756A756E6F120833353830313735370AB6020A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374128D020A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070123F6A756E6F31357533647437397436737878613378336B706B687A737935366564616135613636777674336B786D756B716A7A3273783068657335736E3338671A9C017B22696E6372656173655F616C6C6F77616E6365223A7B22616D6F756E74223A223132373435373637313437222C227370656E646572223A226A756E6F31376834736770616B737967646E737770783734737A76393868676764646330386E79347A7635636136336A763533717074787273686163766B75222C2265787069726573223A7B2261745F686569676874223A31303030303030307D7D7D0AE8010A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E747261637412BF010A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070123F6A756E6F31376834736770616B737967646E737770783734737A76393868676764646330386E79347A7635636136336A763533717074787273686163766B751A4F7B2273776170223A7B22696E7075745F746F6B656E223A22546F6B656E31222C22696E7075745F616D6F756E74223A223132373435373637313437222C226D696E5F6F7574707574223A2231227D7D0AB7020A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374128E020A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070123F6A756E6F31656C3672666D7A366839707770646C66366B32716634647774337935777164376B337870797679746B6C736E6B743975763261716538617134761A4C7B2273776170223A7B22696E7075745F746F6B656E223A22546F6B656E32222C22696E7075745F616D6F756E74223A223637343030353937222C226D696E5F6F7574707574223A2231227D7D2A500A446962632F45443037413333393141313132423137353931354344384641463433413244413845343739304544453132353636363439443043324639373731364238353138120836373430303539370A8D010A1C2F636F736D6F732E62616E6B2E763162657461312E4D736753656E64126D0A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070122B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C3330701A110A05756A756E6F1208353538353135303912680A500A460A1F2F636F736D6F732E63727970746F2E736563703235366B312E5075624B657912230A2102873A070A5DAA2C3431169053489E817F22A5C17C60A5C8409FE775BD88C16F3F12040A020801180612140A0D0A05756A756E6F12043735303310C08DB7011A40EAAE8C0D4939A258ECC7B726EB9D560B13CC070F1D4A3A814B2C8B4270E7BD1F3B594CACDB915F8E405FF5B5E16019765B1819C59180C00063C67DA977CEF075"

In [6]:
cosmos_tx_pb2.Tx().FromString("0AF50A0A89010A1C2F636F736D6F732E62616E6B2E763162657461312E4D736753656E6412690A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070122B6A756E6F313067306C3368643973617533766E6A7261796A68657267637078656D756378637370676E6E341A0D0A05756A756E6F1204313030300AF8010A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E747261637412CF010A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070123F6A756E6F31323464307A796D726B64787637326363797571727175757238646B65736D786D7832756E666E3764656A393579717835796E387337307833796A1A4C7B2273776170223A7B22696E7075745F746F6B656E223A22546F6B656E31222C22696E7075745F616D6F756E74223A223335383031373537222C226D696E5F6F7574707574223A2231227D7D2A110A05756A756E6F120833353830313735370AB6020A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374128D020A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070123F6A756E6F31357533647437397436737878613378336B706B687A737935366564616135613636777674336B786D756B716A7A3273783068657335736E3338671A9C017B22696E6372656173655F616C6C6F77616E6365223A7B22616D6F756E74223A223132373435373637313437222C227370656E646572223A226A756E6F31376834736770616B737967646E737770783734737A76393868676764646330386E79347A7635636136336A763533717074787273686163766B75222C2265787069726573223A7B2261745F686569676874223A31303030303030307D7D7D0AE8010A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E747261637412BF010A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070123F6A756E6F31376834736770616B737967646E737770783734737A76393868676764646330386E79347A7635636136336A763533717074787273686163766B751A4F7B2273776170223A7B22696E7075745F746F6B656E223A22546F6B656E31222C22696E7075745F616D6F756E74223A223132373435373637313437222C226D696E5F6F7574707574223A2231227D7D0AB7020A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374128E020A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070123F6A756E6F31656C3672666D7A366839707770646C66366B32716634647774337935777164376B337870797679746B6C736E6B743975763261716538617134761A4C7B2273776170223A7B22696E7075745F746F6B656E223A22546F6B656E32222C22696E7075745F616D6F756E74223A223637343030353937222C226D696E5F6F7574707574223A2231227D7D2A500A446962632F45443037413333393141313132423137353931354344384641463433413244413845343739304544453132353636363439443043324639373731364238353138120836373430303539370A8D010A1C2F636F736D6F732E62616E6B2E763162657461312E4D736753656E64126D0A2B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C333070122B6A756E6F3134773576326C363533677130683268356B677335766C683764366D6A61686E3237356C3330701A110A05756A756E6F1208353538353135303912680A500A460A1F2F636F736D6F732E63727970746F2E736563703235366B312E5075624B657912230A2102873A070A5DAA2C3431169053489E817F22A5C17C60A5C8409FE775BD88C16F3F12040A020801180612140A0D0A05756A756E6F12043735303310C08DB7011A40EAAE8C0D4939A258ECC7B726EB9D560B13CC070F1D4A3A814B2C8B4270E7BD1F3B594CACDB915F8E405FF5B5E16019765B1819C59180C00063C67DA977CEF075")

NameError: name 'cosmos_tx_pb2' is not defined

In [4]:
from base64 import b64encode
b64_str = b64encode(bytes)

In [5]:
b64_str

b'MEFGNTBBMEE4OTAxMEExQzJGNjM2RjczNkQ2RjczMkU2MjYxNkU2QjJFNzYzMTYyNjU3NDYxMzEyRTRENzM2NzUzNjU2RTY0MTI2OTBBMkI2QTc1NkU2RjMxMzQ3NzM1NzYzMjZDMzYzNTMzNjc3MTMwNjgzMjY4MzU2QjY3NzMzNTc2NkM2ODM3NjQzNjZENkE2MTY4NkUzMjM3MzU2QzMzMzA3MDEyMkI2QTc1NkU2RjMxMzA2NzMwNkMzMzY4NjQzOTczNjE3NTMzNzY2RTZBNzI2MTc5NkE2ODY1NzI2NzYzNzA3ODY1NkQ3NTYzNzg2MzczNzA2NzZFNkUzNDFBMEQwQTA1NzU2QTc1NkU2RjEyMDQzMTMwMzAzMDBBRjgwMTBBMjQyRjYzNkY3MzZENzc2MTczNkQyRTc3NjE3MzZEMkU3NjMxMkU0RDczNjc0NTc4NjU2Mzc1NzQ2NTQzNkY2RTc0NzI2MTYzNzQxMkNGMDEwQTJCNkE3NTZFNkYzMTM0NzczNTc2MzI2QzM2MzUzMzY3NzEzMDY4MzI2ODM1NkI2NzczMzU3NjZDNjgzNzY0MzY2RDZBNjE2ODZFMzIzNzM1NkMzMzMwNzAxMjNGNkE3NTZFNkYzMTMyMzQ2NDMwN0E3OTZENzI2QjY0Nzg3NjM3MzI2MzYzNzk3NTcxNzI3MTc1NzU3MjM4NjQ2QjY1NzM2RDc4NkQ3ODMyNzU2RTY2NkUzNzY0NjU2QTM5MzU3OTcxNzgzNTc5NkUzODczMzczMDc4MzM3OTZBMUE0QzdCMjI3Mzc3NjE3MDIyM0E3QjIyNjk2RTcwNzU3NDVGNzQ2RjZCNjU2RTIyM0EyMjU0NkY2QjY1NkUzMTIyMkMyMjY5NkU3MDc1NzQ1RjYxNkQ2Rjc1NkU3NDIyM0EyMjMzMzUzODMwMzEzNzM1MzcyMjJDMjI2RDY5NkU1RjZGNzU3NDcwNzU3NDIyM0EyMjMxMjI3RD

In [9]:
cryptb_tx = b64decode("Ct8MCokBChwvY29zbW9zLmJhbmsudjFiZXRhMS5Nc2dTZW5kEmkKK2p1bm8xNHc1djJsNjUzZ3EwaDJoNWtnczV2bGg3ZDZtamFobjI3NWwzMHASK2p1bm8xMGcwbDNoZDlzYXUzdm5qcmF5amhlcmdjcHhlbXVjeGNzcGdubjQaDQoFdWp1bm8SBDEwMDAKggIKJC9jb3Ntd2FzbS53YXNtLnYxLk1zZ0V4ZWN1dGVDb250cmFjdBLZAQoranVubzE0dzV2Mmw2NTNncTBoMmg1a2dzNXZsaDdkNm1qYWhuMjc1bDMwcBI/anVubzFhYWd3M204OXplOTR2NmU1NmswZWVlbWxkcms4YWptdGNjbXdlcjlmZnJ6ZHV3M3p1d2RzMHc1OHhrGld7InN3YXAiOnsib2ZmZXJfYXNzZXQiOnsiYW1vdW50IjoiNzQyMTM3MCIsImluZm8iOnsibmF0aXZlX3Rva2VuIjp7ImRlbm9tIjoidWp1bm8ifX19fX0qEAoFdWp1bm8SBzc0MjEzNzAKsgIKJC9jb3Ntd2FzbS53YXNtLnYxLk1zZ0V4ZWN1dGVDb250cmFjdBKJAgoranVubzE0dzV2Mmw2NTNncTBoMmg1a2dzNXZsaDdkNm1qYWhuMjc1bDMwcBI/anVubzFkZDBrMHVtNXJxbmNmdWV6YTYydzlzZW50ZGZoM2VjNG53NGFxNGxrNWhramw2M3ZsanFzY3RoOWd2GpgBeyJpbmNyZWFzZV9hbGxvd2FuY2UiOnsiYW1vdW50IjoiNjk3NDcxMiIsInNwZW5kZXIiOiJqdW5vMWd4dmNsdGtsMHRmMjBycHNuMndmOXE2ZXgwdnI1eGs2ajN0emV6dXY2eXlqZXo5N3c1dnFteGswY3YiLCJleHBpcmVzIjp7ImF0X2hlaWdodCI6MTAwMDAwMDB9fX0K5AEKJC9jb3Ntd2FzbS53YXNtLnYxLk1zZ0V4ZWN1dGVDb250cmFjdBK7AQoranVubzE0dzV2Mmw2NTNncTBoMmg1a2dzNXZsaDdkNm1qYWhuMjc1bDMwcBI/anVubzFneHZjbHRrbDB0ZjIwcnBzbjJ3ZjlxNmV4MHZyNXhrNmozdHplenV2Nnl5amV6OTd3NXZxbXhrMGN2Gkt7InN3YXAiOnsiaW5wdXRfdG9rZW4iOiJUb2tlbjEiLCJpbnB1dF9hbW91bnQiOiI2OTc0NzEyIiwibWluX291dHB1dCI6IjEifX0KtQIKJC9jb3Ntd2FzbS53YXNtLnYxLk1zZ0V4ZWN1dGVDb250cmFjdBKMAgoranVubzE0dzV2Mmw2NTNncTBoMmg1a2dzNXZsaDdkNm1qYWhuMjc1bDMwcBI/anVubzE1dTNkdDc5dDZzeHhhM3gza3BraHpzeTU2ZWRhYTVhNjZ3dnQza3htdWtxanoyc3gwaGVzNXNuMzhnGpsBeyJpbmNyZWFzZV9hbGxvd2FuY2UiOnsiYW1vdW50IjoiMTk4NDc4MDUxOCIsInNwZW5kZXIiOiJqdW5vMTI0ZDB6eW1ya2R4djcyY2N5dXFycXV1cjhka2VzbXhteDJ1bmZuN2Rlajk1eXF4NXluOHM3MHgzeWoiLCJleHBpcmVzIjp7ImF0X2hlaWdodCI6MTAwMDAwMDB9fX0K5wEKJC9jb3Ntd2FzbS53YXNtLnYxLk1zZ0V4ZWN1dGVDb250cmFjdBK+AQoranVubzE0dzV2Mmw2NTNncTBoMmg1a2dzNXZsaDdkNm1qYWhuMjc1bDMwcBI/anVubzEyNGQwenltcmtkeHY3MmNjeXVxcnF1dXI4ZGtlc214bXgydW5mbjdkZWo5NXlxeDV5bjhzNzB4M3lqGk57InN3YXAiOnsiaW5wdXRfdG9rZW4iOiJUb2tlbjIiLCJpbnB1dF9hbW91bnQiOiIxOTg0NzgwNTE4IiwibWluX291dHB1dCI6IjEifX0KjQEKHC9jb3Ntb3MuYmFuay52MWJldGExLk1zZ1NlbmQSbQoranVubzE0dzV2Mmw2NTNncTBoMmg1a2dzNXZsaDdkNm1qYWhuMjc1bDMwcBIranVubzE0dzV2Mmw2NTNncTBoMmg1a2dzNXZsaDdkNm1qYWhuMjc1bDMwcBoRCgV1anVubxIINTU4MzUyMDUScwpOCkYKHy9jb3Ntb3MuY3J5cHRvLnNlY3AyNTZrMS5QdWJLZXkSIwohAoc6Bwpdqiw0MRaQU0iegX8ipcF8YKXIQJ/ndb2IwW8/EgQKAggBEiEKGgoFdWp1bm8SETc1MDMuMDAwMDAwMDAwMDAxEMCNtwEaQJI0xsHiw8JrRV+9VjIS24rUW3nS9PqyOatWblhQSif6H4lgx8IY4FVhM0xjjj2FLVm7NkBsXtZBp5zIyxBgqs0=")
cryptb_tx = cosmos_tx_pb2.Tx().FromString(cryptb_tx)

In [ ]:
cosmos_tx_pb2.Tx().FromString(cryptb_tx)

In [10]:
cryptb_tx

body {
  messages {
    [/cosmos.bank.v1beta1.MsgSend] {
      from_address: "juno14w5v2l653gq0h2h5kgs5vlh7d6mjahn275l30p"
      to_address: "juno10g0l3hd9sau3vnjrayjhergcpxemucxcspgnn4"
      amount {
        denom: "ujuno"
        amount: "1000"
      }
    }
  }
  messages {
    [/cosmwasm.wasm.v1.MsgExecuteContract] {
      sender: "juno14w5v2l653gq0h2h5kgs5vlh7d6mjahn275l30p"
      contract: "juno1aagw3m89ze94v6e56k0eeemldrk8ajmtccmwer9ffrzduw3zuwds0w58xk"
      msg: "{\"swap\":{\"offer_asset\":{\"amount\":\"7421370\",\"info\":{\"native_token\":{\"denom\":\"ujuno\"}}}}}"
      funds {
        denom: "ujuno"
        amount: "7421370"
      }
    }
  }
  messages {
    [/cosmwasm.wasm.v1.MsgExecuteContract] {
      sender: "juno14w5v2l653gq0h2h5kgs5vlh7d6mjahn275l30p"
      contract: "juno1dd0k0um5rqncfueza62w9sentdfh3ec4nw4aq4lk5hkjl63vljqscth9gv"
      msg: "{\"increase_allowance\":{\"amount\":\"6974712\",\"spender\":\"juno1gxvcltkl0tf20rpsn2wf9q6ex0vr5xk6j3tzezuv6yyjez97w5vqmxk0c

In [78]:
frontrun_bounced_tx = b64decode("CsILCokBChwvY29zbW9zLmJhbmsudjFiZXRhMS5Nc2dTZW5kEmkKK2p1bm8xemhxcmZ1OXczc3Vnd3lrZWYzcnE4dDB2bHhrejcydndubnB0dHMSK2p1bm8xMGcwbDNoZDlzYXUzdm5qcmF5amhlcmdjcHhlbXVjeGNzcGdubjQaDQoFdWp1bm8SBDEwMDAK8AEKJC9jb3Ntd2FzbS53YXNtLnYxLk1zZ0V4ZWN1dGVDb250cmFjdBLHAQoranVubzF6aHFyZnU5dzNzdWd3eWtlZjNycTh0MHZseGt6NzJ2d25ucHR0cxI/anVubzFzZzZjaG1rdHVoeWo0bHNyeHJyZGZsZW03Z3NuazRlanY2emtjYzRkM3ZjcXVsenA1NXdzZjRsNGdsGkt7InN3YXAiOiB7ImlucHV0X3Rva2VuIjogIlRva2VuMSIsICJpbnB1dF9hbW91bnQiOiAiMCIsICJtaW5fb3V0cHV0IjogIjEifX0qCgoFdWp1bm8SATAKhgMKJC9jb3Ntd2FzbS53YXNtLnYxLk1zZ0V4ZWN1dGVDb250cmFjdBLdAgoranVubzF6aHFyZnU5dzNzdWd3eWtlZjNycTh0MHZseGt6NzJ2d25ucHR0cxI/anVubzFydzVjZHdsNmN4dzg4MmN6YW40eDJ5c3AzMHpzbTk0bDZqY3FlYWxnODJ2bTJkbDk4OW5zNjN2ZWg4GpwBeyJzd2FwIjogeyJvZmZlcl9hc3NldCI6IHsiYW1vdW50IjogIjc4NDQ3MyIsICJpbmZvIjogeyJuYXRpdmVfdG9rZW4iOiB7ImRlbm9tIjogImliYy9DNENGRjQ2RkQ2REUzNUNBNENGNENFMDMxRTY0M0M4RkRDOUJBNEI5OUFFNTk4RTlCMEVEOThGRTNBMjMxOUY5In19fX19Kk4KRGliYy9DNENGRjQ2RkQ2REUzNUNBNENGNENFMDMxRTY0M0M4RkRDOUJBNEI5OUFFNTk4RTlCMEVEOThGRTNBMjMxOUY5EgY3ODQ0NzMKuQIKJC9jb3Ntd2FzbS53YXNtLnYxLk1zZ0V4ZWN1dGVDb250cmFjdBKQAgoranVubzF6aHFyZnU5dzNzdWd3eWtlZjNycTh0MHZseGt6NzJ2d25ucHR0cxI/anVubzF3d25oa2FndmNkM3RqejZmOHZzZHN3NXBscW53OHF5MmFqM3JyaHFyMmF4dmt0enY5cTJxejhqeG4zGp8BeyJpbmNyZWFzZV9hbGxvd2FuY2UiOiB7ImFtb3VudCI6ICIzOTg0NDgwIiwgInNwZW5kZXIiOiAianVubzE3djJkMjk5M21lNTBlNmRneng1MGNrdXVhaDB2bWZ5YW5sMHNlZ3hzZGNnM3M0cXpxZXJzeXJ2dThuIiwgImV4cGlyZXMiOiB7ImF0X2hlaWdodCI6IDEwMDAwMDAwfX19CuoBCiQvY29zbXdhc20ud2FzbS52MS5Nc2dFeGVjdXRlQ29udHJhY3QSwQEKK2p1bm8xemhxcmZ1OXczc3Vnd3lrZWYzcnE4dDB2bHhrejcydndubnB0dHMSP2p1bm8xN3YyZDI5OTNtZTUwZTZkZ3p4NTBja3V1YWgwdm1meWFubDBzZWd4c2RjZzNzNHF6cWVyc3lydnU4bhpReyJzd2FwIjogeyJpbnB1dF90b2tlbiI6ICJUb2tlbjEiLCAiaW5wdXRfYW1vdW50IjogIjM5ODQ0ODAiLCAibWluX291dHB1dCI6ICIxIn19Co4BChwvY29zbW9zLmJhbmsudjFiZXRhMS5Nc2dTZW5kEm4KK2p1bm8xemhxcmZ1OXczc3Vnd3lrZWYzcnE4dDB2bHhrejcydndubnB0dHMSK2p1bm8xemhxcmZ1OXczc3Vnd3lrZWYzcnE4dDB2bHhrejcydndubnB0dHMaEgoFdWp1bm8SCTU2MTY3Mzc0NBJnClAKRgofL2Nvc21vcy5jcnlwdG8uc2VjcDI1NmsxLlB1YktleRIjCiEDSBRsPVsf9mJnKiwKlTTMOY7SDkaNepv9WGVj575vFpUSBAoCCAEYZhITCg0KBXVqdW5vEgQxMDAwEMCEPRpATcQcY28JreKxH3Jia8aiDmeLZglcdnAqpEkyN759I2oHXm0rk5uzA2StPXu/jySt2w0ix8YxeiqE5MM5Wp+w4Q==")

In [81]:
cosmos_tx_pb2.Tx().FromString(frontrun_bounced_tx)

body {
  messages {
    [/cosmos.bank.v1beta1.MsgSend] {
      from_address: "juno1zhqrfu9w3sugwykef3rq8t0vlxkz72vwnnptts"
      to_address: "juno10g0l3hd9sau3vnjrayjhergcpxemucxcspgnn4"
      amount {
        denom: "ujuno"
        amount: "1000"
      }
    }
  }
  messages {
    [/cosmwasm.wasm.v1.MsgExecuteContract] {
      sender: "juno1zhqrfu9w3sugwykef3rq8t0vlxkz72vwnnptts"
      contract: "juno1sg6chmktuhyj4lsrxrrdflem7gsnk4ejv6zkcc4d3vcqulzp55wsf4l4gl"
      msg: "{\"swap\": {\"input_token\": \"Token1\", \"input_amount\": \"0\", \"min_output\": \"1\"}}"
      funds {
        denom: "ujuno"
        amount: "0"
      }
    }
  }
  messages {
    [/cosmwasm.wasm.v1.MsgExecuteContract] {
      sender: "juno1zhqrfu9w3sugwykef3rq8t0vlxkz72vwnnptts"
      contract: "juno1rw5cdwl6cxw882czan4x2ysp30zsm94l6jcqealg82vm2dl989ns63veh8"
      msg: "{\"swap\": {\"offer_asset\": {\"amount\": \"784473\", \"info\": {\"native_token\": {\"denom\": \"ibc/C4CFF46FD6DE35CA4CF4CE031E643C8FDC9BA4B99AE

In [69]:
with open('contracts.json') as f:
    contracts = json.load(f)

rpc_url = "https://rpc-juno-ia.cosmosia.notional.ventures/"

for contract, info in contracts.items():
    if contract == "juno1mz8yzrgyp9mmq9aksxgpy83vmu8p4j8h3qyf9waxcd2epchqx5ps0ekj27":
        print(contract)
    if info['dex'] == "junoswap":

        pool_info = await junoswap_info(rpc_url, contract)

        token1_denom, = pool_info['token1_denom'].values()
        token1_type, = pool_info['token1_denom'].keys()
        token2_denom, = pool_info['token2_denom'].values()
        token2_type, = pool_info['token2_denom'].keys()
        
        info = {"token1_denom": token1_denom, "token1_type": token1_type, "token2_denom": token2_denom, "token2_type": token2_type}

        contracts[contract]["info"] = info
    
    elif info['dex'] == 'loop':

        pool_info = await terraswap_info(rpc_url, contract)
        if contract == "juno1mz8yzrgyp9mmq9aksxgpy83vmu8p4j8h3qyf9waxcd2epchqx5ps0ekj27":
            print(pool_info)

        info = {}

        if 'token' in pool_info['assets'][0]['info']:
            info['token1_denom'] = pool_info['assets'][0]['info']['token']['contract_addr']
            info['token1_type'] = 'cw20'
        else:
            info['token1_denom'] = pool_info['assets'][0]['info']['native_token']['denom']
            info['token1_type'] = 'native'

        if 'token' in pool_info['assets'][1]['info']:
            info['token2_denom'] = pool_info['assets'][1]['info']['token']['contract_addr']
            info['token2_type'] = 'cw20'
        else:
            info['token2_denom'] = pool_info['assets'][1]['info']['native_token']['denom']
            info['token2_type'] = 'native'
                
        contracts[contract]["info"] = info

juno1mz8yzrgyp9mmq9aksxgpy83vmu8p4j8h3qyf9waxcd2epchqx5ps0ekj27
{'assets': [{'info': {'token': {'contract_addr': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup'}}, 'amount': '154797975472'}, {'info': {'token': {'contract_addr': 'juno14lycavan8gvpjn97aapzvwmsj8kyrvf644p05r0hu79namyj3ens87650k'}}, 'amount': '102274842043'}], 'total_share': '124569525818'}


In [70]:
contracts

{'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07': {'info': {'token1_denom': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup',
   'token1_type': 'cw20',
   'token2_denom': 'ujuno',
   'token2_type': 'native'},
  'dex': 'loop',
  'routes': [['juno124d0zymrkdxv72ccyuqrquur8dkesmxmx2unfn7dej95yqx5yn8s70x3yj',
    'juno1qt7uzjg9su3mk78jpt695rmxce4sa7evz7wa0edexjrsxghy35hsgje5l9',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1sg6chmktuhyj4lsrxrrdflem7gsnk4ejv6zkcc4d3vcqulzp55wsf4l4gl',
    'juno1dl8zdygs57lcwm3e9tnq4pdmmp8nhhf08jcne72nsjkf7qntfz3s0e99x3',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1ctsmp54v79x7ea970zejlyws50cj9pkrmw49x46085fn80znjmpqz2n642',
    'juno1utkr0ep06rkxgsesq6uryug93daklyd6wneesmtvxjkz0xjlte9qdj2s8q',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1jz50fj5zkcv3h6hmcfr3nr6eer7rj5pmsry5qj5jc8rfvpeavyzsgknm83',
    'juno1mz8yzr

In [6]:
contract_list = list(contracts.keys())

In [4]:
RPC_URL = "https://rpc-juno-ia.cosmosia.notional.ventures/"

async def update_reserves(contract_address: str) -> dict:
    print("querying contract")
    if contracts[contract_address]["dex"] == "junoswap":
        contract_info = await junoswap_info(RPC_URL, contract_address)
        contracts[contract_address]["info"]["token1_reserves"] = int(contract_info['token1_reserve'])
        contracts[contract_address]["info"]["token2_reserves"] = int(contract_info['token2_reserve'])
    else:
        contract_info = await terraswap_info(RPC_URL, contract_address)
        contracts[contract_address]["info"]["token1_reserves"] = int(contract_info['assets'][0]['amount'])
        contracts[contract_address]["info"]["token2_reserves"] = int(contract_info['assets'][1]['amount'])

In [9]:
# Update all pool info
await aiometer.run_on_each(update_reserves, contract_list)

querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract
querying contract


In [8]:
contracts

{'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07': {'info': {'token1_denom': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup',
   'token1_type': 'cw20',
   'token2_denom': 'ujuno',
   'token2_type': 'native',
   'token1_reserves': 1626682297412,
   'token2_reserves': 18130594773},
  'dex': 'loop',
  'routes': [['juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07',
    'juno1qt7uzjg9su3mk78jpt695rmxce4sa7evz7wa0edexjrsxghy35hsgje5l9',
    'juno124d0zymrkdxv72ccyuqrquur8dkesmxmx2unfn7dej95yqx5yn8s70x3yj'],
   ['juno1sg6chmktuhyj4lsrxrrdflem7gsnk4ejv6zkcc4d3vcqulzp55wsf4l4gl',
    'juno1dl8zdygs57lcwm3e9tnq4pdmmp8nhhf08jcne72nsjkf7qntfz3s0e99x3',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1ctsmp54v79x7ea970zejlyws50cj9pkrmw49x46085fn80znjmpqz2n642',
    'juno1utkr0ep06rkxgsesq6uryug93daklyd6wneesmtvxjkz0xjlte9qdj2s8q',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno124d0z

In [8]:
pool_info

{'assets': [{'info': {'token': {'contract_addr': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup'}},
   'amount': '1629423315207'},
  {'info': {'native_token': {'denom': 'ujuno'}}, 'amount': '18099944885'}],
 'total_share': '166673312855'}

In [71]:
with open('contracts_2.json', 'w') as f:
    json.dump(contracts, f, indent=4)

In [72]:
with open('contracts_2.json') as f:
    contracts = json.load(f)

In [18]:
contracts

{'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07': {'info': {'token1_denom': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup',
   'token1_type': 'cw20',
   'token2_denom': 'ujuno',
   'token2_type': 'native'},
  'dex': 'loop',
  'routes': [['juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07',
    'juno1qt7uzjg9su3mk78jpt695rmxce4sa7evz7wa0edexjrsxghy35hsgje5l9',
    'juno124d0zymrkdxv72ccyuqrquur8dkesmxmx2unfn7dej95yqx5yn8s70x3yj'],
   ['juno1sg6chmktuhyj4lsrxrrdflem7gsnk4ejv6zkcc4d3vcqulzp55wsf4l4gl',
    'juno1dl8zdygs57lcwm3e9tnq4pdmmp8nhhf08jcne72nsjkf7qntfz3s0e99x3',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1ctsmp54v79x7ea970zejlyws50cj9pkrmw49x46085fn80znjmpqz2n642',
    'juno1utkr0ep06rkxgsesq6uryug93daklyd6wneesmtvxjkz0xjlte9qdj2s8q',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno124d0zymrkdxv72ccyuqrquur8dkesmxmx2unfn7dej95yqx5yn8s70x3yj',
    'juno1qt7uzj

In [73]:
contracts_symmetric = contracts.copy()
for contract in contracts:
    route_set_list = []
    routes_symmetric = []
    for route in contracts[contract]['routes']:
        if set(route) not in route_set_list:
            route_set_list.append(set(route))
            routes_symmetric.append(route)
        else:
            pass
    contracts_symmetric[contract]["routes"] = routes_symmetric

In [20]:
with open('contracts_2.json', 'w') as f:
    json.dump(contracts_symmetric, f, indent=4)

In [74]:
with open('contracts_2.json') as f:
    contracts = json.load(f)

In [75]:
contracts

{'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07': {'info': {'token1_denom': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup',
   'token1_type': 'cw20',
   'token2_denom': 'ujuno',
   'token2_type': 'native'},
  'dex': 'loop',
  'routes': [['juno124d0zymrkdxv72ccyuqrquur8dkesmxmx2unfn7dej95yqx5yn8s70x3yj',
    'juno1qt7uzjg9su3mk78jpt695rmxce4sa7evz7wa0edexjrsxghy35hsgje5l9',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1sg6chmktuhyj4lsrxrrdflem7gsnk4ejv6zkcc4d3vcqulzp55wsf4l4gl',
    'juno1dl8zdygs57lcwm3e9tnq4pdmmp8nhhf08jcne72nsjkf7qntfz3s0e99x3',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1ctsmp54v79x7ea970zejlyws50cj9pkrmw49x46085fn80znjmpqz2n642',
    'juno1utkr0ep06rkxgsesq6uryug93daklyd6wneesmtvxjkz0xjlte9qdj2s8q',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1jz50fj5zkcv3h6hmcfr3nr6eer7rj5pmsry5qj5jc8rfvpeavyzsgknm83',
    'juno1mz8yzr

In [62]:
# order routes

for contract in contracts:
    print(contract)
    routes = []
    for route in contracts[contract]['routes']:
        print(contracts[route[1]]['info'])

        if contracts[route[0]]['info']['token1_denom'] == 'ujuno':
            pool1_out_denom = contracts[route[0]]['info']['token2_denom']
            if contract == route[0]:
                out_denom = contracts[route[0]]['info']['token2_denom']
                in_denom = contracts[route[0]]['info']['token1_denom']
                in_token = "token1"
        else:
            pool1_out_denom = contracts[route[0]]['info']['token1_denom']
            if contract == route[0]:
                out_denom = contracts[route[0]]['info']['token1_denom']
                in_denom = contracts[route[0]]['info']['token2_denom']
                in_token = "token2"

        if contracts[route[1]]['info']['token1_denom'] == pool1_out_denom:
            pool2_out_denom = contracts[route[1]]['info']['token2_denom']
            if contract == route[1]:
                in_token = "token1"
                out_denom = contracts[route[1]]['info']['token2_denom']
                in_denom = contracts[route[1]]['info']['token1_denom']
        else:
            pool2_out_denom = contracts[route[1]]['info']['token1_denom']
            if contract == route[0]:
                out_denom = contracts[route[0]]['info']['token1_denom']
                in_denom = contracts[route[0]]['info']['token2_denom']
                in_token = "token2"

        if contracts[route[2]]['info']['token1_denom'] == 'ujuno':
            if contract == route[2]:
                in_token = "token2"
                out_denom = contracts[route[2]]['info']['token1_denom']
                in_denom = contracts[route[2]]['info']['token2_denom']
        else:
            if contract == route[2]:
                in_token = "token1"
                out_denom = contracts[route[2]]['info']['token2_denom']
                in_denom = contracts[route[2]]['info']['token1_denom']

        contracts[contract]["in_token"] = in_token
        contracts[contract]["out_token"] = "token2" if in_token == "token1" else "token1"

        contracts[contract]["in_denom"] = in_denom
        contracts[contract]["out_denom"] = out_denom

        if "in_denom" not in contracts[contract]:
            routes.append(route)
        else:
            if contracts[contract]["in_denom"] == pool1_out_denom:
                routes.append(route)
            else:
                route.reverse()
                routes.append(route)
    contracts[contract]["routes"] = routes

juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07
{'token1_denom': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup', 'token1_type': 'cw20', 'token2_denom': 'juno15u3dt79t6sxxa3x3kpkhzsy56edaa5a66wvt3kxmukqjz2sx0hes5sn38g', 'token2_type': 'cw20'}
{'token1_denom': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup', 'token1_type': 'cw20', 'token2_denom': 'ibc/C4CFF46FD6DE35CA4CF4CE031E643C8FDC9BA4B99AE598E9B0ED98FE3A2319F9', 'token2_type': 'native'}
{'token1_denom': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup', 'token1_type': 'cw20', 'token2_denom': 'ibc/EAC38D55372F38F1AFD68DF7FE9EF762DCF69F26520643CF3F9D292A738D8034', 'token2_type': 'native'}
{'token1_denom': 'juno14lycavan8gvpjn97aapzvwmsj8kyrvf644p05r0hu79namyj3ens87650k', 'token1_type': 'cw20'}


KeyError: 'token2_denom'

In [51]:
contracts

{'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07': {'info': {'token1_denom': 'juno1qsrercqegvs4ye0yqg93knv73ye5dc3prqwd6jcdcuj8ggp6w0us66deup',
   'token1_type': 'cw20',
   'token2_denom': 'ujuno',
   'token2_type': 'native'},
  'dex': 'loop',
  'routes': [['juno124d0zymrkdxv72ccyuqrquur8dkesmxmx2unfn7dej95yqx5yn8s70x3yj',
    'juno1qt7uzjg9su3mk78jpt695rmxce4sa7evz7wa0edexjrsxghy35hsgje5l9',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1sg6chmktuhyj4lsrxrrdflem7gsnk4ejv6zkcc4d3vcqulzp55wsf4l4gl',
    'juno1dl8zdygs57lcwm3e9tnq4pdmmp8nhhf08jcne72nsjkf7qntfz3s0e99x3',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1ctsmp54v79x7ea970zejlyws50cj9pkrmw49x46085fn80znjmpqz2n642',
    'juno1utkr0ep06rkxgsesq6uryug93daklyd6wneesmtvxjkz0xjlte9qdj2s8q',
    'juno1qc8mrs3hmxm0genzrd92akja5r0v7mfm6uuwhktvzphhz9ygkp8ssl4q07'],
   ['juno1jz50fj5zkcv3h6hmcfr3nr6eer7rj5pmsry5qj5jc8rfvpeavyzsgknm83',
    'juno1mz8yzr

In [52]:
with open('contracts.json', 'w') as f:
    json.dump(contracts, f, indent=4)

In [54]:
lol = [[1,2,3], [4,5,6]]

In [55]:
for lol_list in lol:
    lol_list.reverse()
    print(lol_list)

[3, 2, 1]
[6, 5, 4]


In [56]:
lol

[[3, 2, 1], [6, 5, 4]]